In [35]:
import pandas as pd
from plot_split_loader import plotwise_split, data_merge, plotwise_kfold, PlotwiseKFold

train1, train2, test1, test2 = plotwise_split()

combined_train = data_merge(train1, train2,
    indicator_features = ["LAT", "LON", "ALIVE_pre_burn", "CULL_pre_burn", "DIA_pre_burn", "HT_pre_burn", "CR_pre_burn",
                         "DRYBIO_AG_pre_burn", "ELEV", "SOFTWOOD", "YRS_SINCE_BURN", "NUM_BURNS",
                         "BURN_AREA_TOTAL"])
combined_test = data_merge(test1, test2,
    indicator_features = ["ALIVE_pre_burn", "CULL_pre_burn", "DIA_pre_burn", "HT_pre_burn", "CR_pre_burn",
                         "DRYBIO_AG_pre_burn", "ELEV", "SOFTWOOD", "YRS_SINCE_BURN", "NUM_BURNS",
                         "BURN_AREA_TOTAL"])

SPCD inconsistent; column will not be merged.
SPGRPCD inconsistent; column will not be merged.
SPCD inconsistent; column will not be merged.
SPGRPCD inconsistent; column will not be merged.


/Users/allierose/Library/CloudStorage/OneDrive-DukeUniversity/erdos/Erdos_Trees/machine_learning/plot_split_loader.py:115: DtypeWarning: Columns (55,59,63) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(trimmed_data_path)


In [36]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold,RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
import scipy.stats
from scipy.stats import uniform, randint
import numpy as np

In [37]:
indicator_features = ["DIA_pre_burn", "HT_pre_burn", "DRYBIO_AG_pre_burn", "CR_pre_burn",
                      "ELEV", "BURN_AREA_TOTAL","NUM_BURNS","YRS_SINCE_BURN"]

In [38]:

Accuracies = np.zeros((5,1))
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Step 1: Standardize features
    ('pca', PCA(n_components=3)),  # Step 2: Reduce dimensionality with PCA
    ('logreg', LogisticRegression())  # Step 3: Logistic Regression
])
for i,(train_index, test_index) in enumerate(plotwise_kfold(combined_train)):
    trees_tt = combined_train.iloc[train_index]
    trees_ho = combined_train.iloc[test_index]

    pipeline.fit(trees_tt[indicator_features],trees_tt['ALIVE_post_burn'])
    pred = pipeline.predict(trees_ho[indicator_features])
    Accuracies[i] = accuracy_score(trees_ho['ALIVE_post_burn'],pred)

print(np.mean(Accuracies))

0.7258402121342273


Let's try tuning the PCA and Logistic Regression Models

In [39]:
plotwise_cv = PlotwiseKFold(train_data=combined_train, n_splits=5)
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Step 1: Standardize features
    ('pca', PCA()),                # Step 2: PCA (no n_components defined yet)
    ('logreg', LogisticRegression(max_iter=1000))  # Step 3: Logistic Regression
])

param_distributions = {
    'pca__n_components': randint(2, 5),  
    'logreg__C': uniform(0.01, 10)}

random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=15,  
    scoring='accuracy',  
    cv=plotwise_cv,               
    random_state=216,verbose=2     
)

random_search.fit(combined_train[indicator_features], combined_train['ALIVE_post_burn'].values)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV] END .logreg__C=0.36589034976299395, pca__n_components=3; total time=   0.0s
[CV] END .logreg__C=0.36589034976299395, pca__n_components=3; total time=   0.0s
[CV] END .logreg__C=0.36589034976299395, pca__n_components=3; total time=   0.0s
[CV] END .logreg__C=0.36589034976299395, pca__n_components=3; total time=   0.0s
[CV] END .logreg__C=0.36589034976299395, pca__n_components=3; total time=   0.0s
[CV] END ...logreg__C=6.866410428393421, pca__n_components=3; total time=   0.0s
[CV] END ...logreg__C=6.866410428393421, pca__n_components=3; total time=   0.0s
[CV] END ...logreg__C=6.866410428393421, pca__n_components=3; total time=   0.0s
[CV] END ...logreg__C=6.866410428393421, pca__n_components=3; total time=   0.0s
[CV] END ...logreg__C=6.866410428393421, pca__n_components=3; total time=   0.0s
[CV] END ..logreg__C=2.4266188573308254, pca__n_components=3; total time=   0.0s
[CV] END ..logreg__C=2.4266188573308254, pca__n_

RandomizedSearchCV(cv=PlotwiseKFold(n_splits=5, random_state=216, shuffle=True,
       train_data=                   CN   PLOT        LAT         LON  ALIVE_pre_burn  \
0       8601094010901  78293  46.007125 -117.618379               1
1       8601102010901  78293  46.007125 -117.618379               1
2       8601101010901  78293  46.007125 -117.618379               1
3    ...            NaN
9711     2.204000e+08                0             0.0

[7486 rows x 17 columns]),
                   estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                             ('pca', PCA()),
                                             ('logreg',
                                              LogisticRegression(max_iter=1000))]),
                   n_iter=15,
                   param_distributions={'logreg__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x179b81250>,
                                        'pca__n_components': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x16f858920>},
                   random_state=216, scoring='accuracy', verbose=2)

In [40]:
cv_results = pd.DataFrame(random_search.cv_results_)
best_kernel = random_search.best_params_
best_index = random_search.best_index_  # Index of the best parameter set
print(cv_results.iloc[best_index])   # Full details of the best set
print(cv_results.iloc[best_index]['params'])   # Full details of the best set

mean_fit_time                                                       0.007168
std_fit_time                                                        0.001399
mean_score_time                                                      0.00194
std_score_time                                                      0.000404
param_logreg__C                                                      0.36589
param_pca__n_components                                                    3
params                     {'logreg__C': 0.36589034976299395, 'pca__n_com...
split0_test_score                                                    0.72802
split1_test_score                                                   0.748663
split2_test_score                                                   0.607094
split3_test_score                                                   0.745822
split4_test_score                                                   0.799602
mean_test_score                                                      0.72584

Mean score of 72.5% :(